### Daten laden und Zusammenführen
In diesem Schritt laden wir die Daten aus den Pickle-Dateien und führen die DataFrames zusammen.


In [1]:
import pandas as pd

# Daten laden
df_occupation = pd.read_pickle('./pickle/naics_occupation.pkl')
df_pattern = pd.read_pickle('./pickle/naics_pattern.pkl')

# Daten zusammenführen
merged_df = pd.merge(df_occupation, df_pattern, on=['FIPS', 'naics'], how='inner')


### Überprüfung der Überschneidungen:

In [2]:
merged_df.head()

,FIPS,State_GEOID_x,naics,NAICS_TITLE,emp_total_county_naics,OCC_CODE,OCC_TITLE,emp_occupation,State_GEOID_y,County_GEOID,...,n20_49,n50_99,n100_249,n250_499,n500_999,n1000,n1000_1,n1000_2,n1000_3,n1000_4
0,13073,13,2373,"Highway, Street, and Bridge Construction",27,11-3051,Industrial Production Managers,0.022497,13,073,...,N,N,N,N,N,N,N,N,N,N
1,13073,13,2381,"Foundation, Structure, and Building Exterior C...",231,11-3051,Industrial Production Managers,0.067490,13,073,...,4,N,N,N,N,N,N,N,N,N
2,13073,13,2382,Building Equipment Contractors,868,11-3051,Industrial Production Managers,0.134981,13,073,...,10,N,N,N,N,N,N,N,N,N
3,13073,13,2383,Building Finishing Contractors,281,11-3051,Industrial Production Managers,0.078739,13,073,...,6,N,N,N,N,N,N,N,N,N
4,13073,13,3211,Sawmills and Wood Preservation,116,11-3051,Industrial Production Managers,0.978609,13,073,...,N,N,N,N,N,N,N,N,N,N


### Umwandlung von Zeichenfolgen in numerische Werte
Die Unternehmensgrößenklassen-Spalten enthalten 'N' für 'Nicht verfügbar'. Diese werden in 0 umgewandelt, und die restlichen Zeichenfolgen werden in Ganzzahlen umgewandelt.


In [3]:
# Daten zusammenführen
merged_df = pd.merge(df_occupation, df_pattern, on=['FIPS', 'naics'], how='inner')

# Funktion zur Umwandlung der Zeichenfolgen in numerische Werte
def convert_to_numeric(series):
    return series.replace('N', 0).astype(int)

# Umwandlung der Zeichenfolgen in den Unternehmensgrößenklassen in numerische Werte
size_columns = ['n<5', 'n5_9', 'n10_19', 'n20_49', 'n50_99', 'n100_249', 'n250_499', 'n500_999', 'n1000', 'n1000_1', 'n1000_2', 'n1000_3', 'n1000_4']
for col in size_columns:
    merged_df[col] = convert_to_numeric(merged_df[col])

### Festlegen der Aggregationsmethoden
Für jede numerische Spalte legen wir fest, wie die Werte aggregiert werden sollen. Summen werden für absolute Zahlen wie Beschäftigungszahlen und Unternehmensgrößenklassen berechnet.

In [4]:
# Festlegen der Aggregationsmethoden
agg_methods = {
    'emp_total_county_naics': 'sum',
    'emp_occupation': 'sum',
    'emp': 'sum',
    'qp1': 'sum',
    'ap': 'sum',
    'est': 'sum',
    'n<5': 'sum',
    'n5_9': 'sum',
    'n10_19': 'sum',
    'n20_49': 'sum',
    'n50_99': 'sum',
    'n100_249': 'sum',
    'n250_499': 'sum',
    'n500_999': 'sum',
    'n1000': 'sum',
    'n1000_1': 'sum',
    'n1000_2': 'sum',
    'n1000_3': 'sum',
    'n1000_4': 'sum',
}

### Aggregation der Daten
Die Daten werden nach dem FIPS-Code gruppiert und gemäß den festgelegten Aggregationsmethoden aggregiert.

In [5]:
# Aggregation der Daten nach FIPS
aggregated_df = merged_df.groupby('FIPS').agg(agg_methods).reset_index()
aggregated_df.head()

,FIPS,emp_total_county_naics,emp_occupation,emp,qp1,ap,est,n<5,n5_9,n10_19,n20_49,n50_99,n100_249,n250_499,n500_999,n1000,n1000_1,n1000_2,n1000_3,n1000_4
0,1001,23060,288.625130,23060,293275,1278447,2806,1363,482,130,32,0,0,0,0,0,0,0,0,0
1,1003,336985,3596.387783,336985,4080329,18026558,25490,13502,4200,2950,1366,241,21,0,0,0,0,0,0,0
2,1005,25322,312.613142,25322,261186,980777,1087,404,40,0,0,0,0,0,0,0,0,0,0,0
3,1007,19611,224.743546,19611,299619,1400868,831,280,63,0,0,54,0,0,0,0,0,0,0,0
4,1009,32406,392.080670,32406,360149,1640600,4143,2401,300,267,117,0,0,0,0,0,0,0,0,0


### Speichern als Pickle

In [6]:
aggregated_df.to_pickle('./pickle/aggregated_df.pkl')